In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
import re

import numpy as np
from scipy.stats import chi2_contingency

### Funções

In [30]:
# MÉTODOS
def extrair_dados_tabela (soup, tag_id):
    produtos = []

    # --- Localiza a tabela com id="tabResult"
    #tabela = soup.find("table", {"id": "tabResult"})
    tabela = soup.find("table", {"id": tag_id})
    
    if tabela:
        linhas = tabela.find_all("tr")
        for linha in linhas:
            colunas = linha.find_all("td")
            if len(colunas) >= 2:
                # Descrição
                descricao_tag = colunas[0].find("span", class_="txtTit")
                descricao = descricao_tag.text.strip() if descricao_tag else ""

                # Código
                codigo_tag = colunas[0].find("span", class_="RCod")
                codigo = ""
                if codigo_tag:
                    match = re.search(r"\d+", codigo_tag.text)
                    if match:
                        codigo = match.group()

                # Extrai o restante do texto do td esquerdo
                info_extra = colunas[0].text

                # Regex para quantidade, unidade e valor unitário
                qtd_match = re.search(r"Qtde\.\s*:\s*([\d,\.]+)", info_extra)
                un_match = re.search(r"UN:\s*([A-Za-z]+)", info_extra)
                vl_unit_match = re.search(r"Vl\. Unit\.\s*:\s*([\d,]+)", info_extra)

                quantidade = qtd_match.group(1) if qtd_match else ""
                unidade = un_match.group(1) if un_match else ""
                valor_unitario = vl_unit_match.group(1) if vl_unit_match else ""

                # Valor total
                valor_tag = colunas[1].find("span", class_="valor")
                valor_total = valor_tag.text.strip() if valor_tag else ""

                produtos.append({
                    "descricao_item": descricao,
                    "cod": codigo,
                    "qtde": quantidade,
                    "unid": unidade,
                    "valor_unitario": valor_unitario,
                    "valor_total": valor_total
                })

        df = pd.DataFrame(produtos)
        #print(df)
        return df
    else:
        print("Tabela 'tabResult' não encontrada.")


def extrair_nome_supermercado(soup, tag_info, df):
    # Obtém o nome da empresa e cnpj
    nome = soup.find(id=tag_info).string
    #supplier_name = soup.find(id="u20").string
    df["supermercado"] = nome 
    return df

def extrair_cnpj_endereco (soup, tag_classe, df):
    # Get the CNPJ Number and Address

    # --- Pega todos os <div class="text">
    divs_text = soup.find_all('div', class_= tag_classe)
    #print(divs_text[1].text) # pode usar o .text para obter os valores

    # Inicializa valores
    cnpj = ""
    endereco = ""

    for i, div in enumerate(divs_text):
        texto = div.text.strip()
        #print(i, text, len(text))
        if 'CNPJ' in texto:
            # Extrai CNPJ com regex
            cnpj_match = re.search(r"\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}", texto)

            if cnpj_match:
                cnpj = cnpj_match.group()
                df["cnpj_emissor"] = cnpj
            
            # Tenta pegar o próximo div como endereço
            if i + 1 < len(divs_text):
                endereco = divs_text[i + 1].text.strip()
                df["endereco"] = endereco
            break

    return df

def extrair_dados_nfe (soup, tag, df): 
    li_info = soup.find(tag) # Localiza o <li> que contém "Número:"
    dados = {}

    for strong in li_info.find_all("strong"):
        label = strong.text.strip().replace(":", "")
        next_text = strong.next_sibling
        if next_text:
            value = next_text.strip()
            dados[label] = value

    # --- Extrai os valores
    df["numero_nfe"] = dados.get("Número", "")
    df["numero_serie"] = dados.get("Série", "")

    emissao_val = dados.get("Emissão", "")
    # --- Remove qualquer coisa após " -", mantendo apenas a data/hora com fuso
    emissao_limpo = emissao_val.split(" -")[0].strip()
    # Converte para datetime
    emissao_datetime = datetime.strptime(emissao_limpo, "%d/%m/%Y %H:%M:%S%z")

    df["datetime_emissao"] = emissao_datetime

    return df

def categorizar_produtos(produto):
    nome_produto = str(produto).lower()

    categorias = {
        'Meat': r'\b(frango|carne|peito|coxa|lingui[çc]a|presunto|bife|pernil)\b',
        'Grains': r'\b(arroz|feij[aã]o|macarr[aã]o|farinha|massa|cuscuz|aveia)\b',
        'Dairy': r'\b(leite|iogurte|queijo|requeij[aã]o|manteiga|creme de leite)\b',
        'Eggs': r'\b(ovo|ovos)\b',
        'Fruits': r'\b(fruta|banana|ma[çc]a|uva|laranja|mam[aã]o|abacaxi|mel[aã]o|melancia)\b',
        'Vegetables': r'\b(alface|tomate|cenoura|batata|cebola|alho|pepino|piment[aã]o|chuchu)\b',
        'Beverages': r'\b(refrigerante|suco|[cç]erveja|vinho|[cç]h[aã]|[cç]afe)\b',
        'Oil': r'\b([oó]leo|azeite|óleo de soja)\b',
        'Snacks': r'\b(salgadinho|biscoito|chocolate|bala|doce|barrinha|pipoca)\b',
        'Cleaning': r'\b(detergente|sab[aã]o|desinfetante|limpador|amaciante)\b',
        'Personal Care': r'\b(sabonete|shampoo|creme dental|escova|papel higi[eé]nico|desodorante)\b',
        'Bakery': r'\b(p[aã]o|bolo|p[aã]o de forma|broa|rosca)\b'
    }

    for categoria, padrao in categorias.items():
        if re.search(padrao, nome_produto):
            return categoria
    
    return 'Outros'


#### 1. Carregamento dos Dados (Web Scrapping)

In [17]:
url_qrcode = "http://nfe.sefaz.ba.gov.br/servicos/nfce/modulos/geral/NFCEC_consulta_chave_acesso.aspx?p=29250306057223048450650150000775861151017568|2|1|1|8DF25CB77517F73F80677AE283AEE7DFCCA38282"

# Configura o Selenium para rodar em modo "headless" (sem abrir janela)
chrome_options = Options()
chrome_options.add_argument("--headless")  # Remova essa linha se quiser ver o navegador
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# Inicializa o navegador
driver = webdriver.Chrome(options=chrome_options)
driver.get(url_qrcode)

# Aguarda a página carregar completamente
time.sleep(3)

# Extrai o HTML renderizado
html = driver.page_source
driver.quit()


In [18]:
# --- Faz o parsing com BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Extrair Dados da Compra "Tabela de itens"
tag_tabela = "tabResult" 
df_raw = extrair_dados_tabela(soup, tag_tabela)

# Extrai nome do supermercado por id
tag_info = "u20"
df = extrair_nome_supermercado(soup, tag_info, df_raw)

tag_id = "text"
df = extrair_cnpj_endereco(soup, tag_id, df)

tag_div = "li"
df = extrair_dados_nfe(soup, tag_div, df)

#### 2. Exploração dos Dados

In [ ]:
# Verifica as 5 primeiras linhas do dataframe
#df.head()

# Verifica o shape
print(f" Nº de Linhas= {df.shape[0]} \n Nº de Colunas= {df.shape[1]}")

# Verifica o tipo de dados das colunas
df.info()

# Verifica valores nulos
df.isnull().sum()

# Verifica a Estatistica Descritiva do Dataframe
df.describe()

#### 3.Limpeza dos Dados 
- Manipular valores nulos, em branco;
- Converter tipo de dados;

In [ ]:
# MISSING VALUES
#valor_medio = df[0].median()
#df[0] = df[0].fillna(valor_medio)

In [19]:
# CONVERT DATA TYPE
# Converte colunas para tipo numérico
cols_to_convert = ['qtde', 'valor_unitario', 'valor_total']

for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col].str.replace(",", "."), errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype                    
---  ------            --------------  -----                    
 0   descricao_item    108 non-null    object                   
 1   cod               108 non-null    object                   
 2   qtde              108 non-null    float64                  
 3   unid              108 non-null    object                   
 4   valor_unitario    108 non-null    float64                  
 5   valor_total       108 non-null    float64                  
 6   supermercado      108 non-null    object                   
 7   cnpj_emissor      108 non-null    object                   
 8   endereco          108 non-null    object                   
 9   numero_nfe        108 non-null    object                   
 10  numero_serie      108 non-null    object                   
 11  datetime_emissao  108 non-null    datetime64[

In [ ]:
# OUTLIERS - NA
# Identifica outliers
q1 = df['valor_total'].quantile(0.25)
q3 = df['valor_total'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filtra outliers
df = df[~((df['valor_total'] < lower_bound) | (df['valor_total'] > upper_bound))]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102 entries, 0 to 107
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype                    
---  ------            --------------  -----                    
 0   descricao_item    102 non-null    object                   
 1   cod               102 non-null    object                   
 2   qtde              102 non-null    float64                  
 3   unid              102 non-null    object                   
 4   valor_unitario    102 non-null    float64                  
 5   valor_total       102 non-null    float64                  
 6   supermercado      102 non-null    object                   
 7   cnpj_emissor      102 non-null    object                   
 8   endereco          102 non-null    object                   
 9   numero_nfe        102 non-null    object                   
 10  numero_serie      102 non-null    object                   
 11  datetime_emissao  102 non-null    datetime64[us, U

In [23]:
# Verifica a Estatistica Descritiva do Dataframe
df.describe()

,qtde,valor_unitario,valor_total
count,102.000000,102.000000,102.000000
mean,1.204824,13.420686,9.190686
std,1.667404,15.778988,7.455157
min,0.050000,1.250000,0.850000
25%,0.793750,3.550000,3.690000
50%,1.000000,7.240000,6.020000
75%,1.000000,16.825000,12.892500
max,12.000000,83.000000,30.980000


#### 4. Data Wrangling

In [40]:
# Create 'total_cost' column (assuming 'date' column represents shopping trips)
df['custo_total'] = df.groupby('datetime_emissao')['valor_total'].transform('sum')

df['categoria'] = df['descricao_item'].apply(categorizar_produtos)

# Aggregate data
df_product_summary = df.groupby(['categoria']).agg(
    qtde_total=('qtde', 'count'),
    preco_unit_medio=('valor_unitario', 'mean'),
    total_gasto=('valor_total', 'sum')
).reset_index()

display(df_product_summary.head(10))

C:\Users\reyna\AppData\Local\Temp\ipykernel_4440\2446088937.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['custo_total'] = df.groupby('datetime_emissao')['valor_total'].transform('sum')
C:\Users\reyna\AppData\Local\Temp\ipykernel_4440\2446088937.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['categoria'] = df['descricao_item'].apply(categorizar_produtos)


,categoria,qtde_total,preco_unit_medio,total_gasto
0,Bakery,2,20.400000,8.94
1,Dairy,1,1.890000,1.89
2,Fruits,5,7.072000,31.07
3,Grains,1,15.900000,15.90
4,Outros,87,13.978276,862.93
5,Vegetables,6,9.808333,16.72


#### 5. DATA ANALYSIS

In [ ]:
# Calculate total spending per product category
spending_per_category = df_product_summary.groupby('categoria')['total_gasto'].sum()
spending_per_category.head()

categoria
Bakery      8.94
Dairy       1.89
Fruits     31.07
Grains     15.90
Outros    862.93
Name: total_gasto, dtype: float64

In [43]:
# Identify the top 5 most expensive products
top_5_products = df_product_summary.nlargest(5, 'total_gasto')
top_5_products.head()

,categoria,qtde_total,preco_unit_medio,total_gasto
4,Outros,87,13.978276,862.93
2,Fruits,5,7.072000,31.07
5,Vegetables,6,9.808333,16.72
3,Grains,1,15.900000,15.90
0,Bakery,2,20.400000,8.94


In [44]:
# Identify the top 5 most expensive product categories
top_5_categories = spending_per_category.nlargest(5)
top_5_categories.head()

categoria
Outros        862.93
Fruits         31.07
Vegetables     16.72
Grains         15.90
Bakery          8.94
Name: total_gasto, dtype: float64

In [45]:
# Calculate the average price per unit, handling potential division by zero
df_product_summary['average_price_per_unit'] = df_product_summary['total_gasto'] / df_product_summary['qtde_total']
df_product_summary['average_price_per_unit'] = df_product_summary['average_price_per_unit'].fillna(0)
df_product_summary.head()

,categoria,qtde_total,preco_unit_medio,total_gasto,average_price_per_unit
0,Bakery,2,20.400000,8.94,4.470000
1,Dairy,1,1.890000,1.89,1.890000
2,Fruits,5,7.072000,31.07,6.214000
3,Grains,1,15.900000,15.90,15.900000
4,Outros,87,13.978276,862.93,9.918736


In [48]:
# Analyze potential correlations between product categories and total spending
# Use Cramer's V for categorical-numerical correlation (as an example)

contingency_table = pd.crosstab(df_product_summary['categoria'], df_product_summary['total_gasto'].astype(int))
chi2, p, dof, expected = chi2_contingency(contingency_table)
cramers_v = np.sqrt(chi2 / (df_product_summary.shape[0] * (min(contingency_table.shape) - 1)))
category_spending_correlation = cramers_v

# Print or display the results
print("Total Spending per Category:\n", spending_per_category)
print("\nTop 5 Most Expensive Products:\n", top_5_products)
print("\nTop 5 Most Expensive Categories:\n", top_5_categories)
display(df_product_summary)
print("\nCramer's V for Category-Spending Correlation:", category_spending_correlation)

Total Spending per Category:
 categoria
Bakery          8.94
Dairy           1.89
Fruits         31.07
Grains         15.90
Outros        862.93
Vegetables     16.72
Name: total_gasto, dtype: float64

Top 5 Most Expensive Products:
     categoria  qtde_total  preco_unit_medio  total_gasto
4      Outros          87         13.978276       862.93
2      Fruits           5          7.072000        31.07
5  Vegetables           6          9.808333        16.72
3      Grains           1         15.900000        15.90
0      Bakery           2         20.400000         8.94

Top 5 Most Expensive Categories:
 categoria
Outros        862.93
Fruits         31.07
Vegetables     16.72
Grains         15.90
Bakery          8.94
Name: total_gasto, dtype: float64


,categoria,qtde_total,preco_unit_medio,total_gasto,average_price_per_unit
0,Bakery,2,20.400000,8.94,4.470000
1,Dairy,1,1.890000,1.89,1.890000
2,Fruits,5,7.072000,31.07,6.214000
3,Grains,1,15.900000,15.90,15.900000
4,Outros,87,13.978276,862.93,9.918736
5,Vegetables,6,9.808333,16.72,2.786667



Cramer's V for Category-Spending Correlation: 1.0000000000000002
